### Compare 2 images with Claude 3 Sonnet

In this notebook we will use Claude 3's vision capabilities to compare to image ID's. 

### 1. Setup Amazon Bedrock

In [70]:
%%capture
#Install dependencies 
!pip3 install -qU boto3

In [71]:
#Imports
import boto3
import json, sys
from datetime import datetime

print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.34.144


In [72]:
#Setting Up Bedrock Client
region = 'us-west-2' #AWS Region
print('Using region: ', region)

bedrock = boto3.client( #Bedrock Client 
    service_name = 'bedrock-runtime',
    region_name = region,
    )
#Model Id (Claude 3 Haiku)
modelId = "anthropic.claude-3-haiku-20240307-v1:0"
#Model Id (Claude 3 Sonnet)
#modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

Using region:  us-east-1


### Open Image files

In [73]:
#Open Image & convert it into Bytes (FakeLicense)
with open("DriverLicense1.png", "rb") as image:
  f = image.read()
  image_byte = bytearray(f)
    
with open("DriverLicense1Fraud.png", "rb") as image:
  f = image.read()
  image_byte1 = bytearray(f)

### Bedrock Converse API Setup

In [74]:
#Set up a function to call the Bedrock Converse API
def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    try:
        response = client.converse(
            modelId=id, #Passing Along modelId
            messages=[ #Messages is where the payload goes
                {
                    "role": "assistant", #Indicating AI input
                    "content": [ #Content of the payload as an object 
                        {
                            "text": "You are an expert analyst at detecting fraudulent ID documents" 
                        }
                    ],
                    "role": "user", #Indicating user input
                    "content": [ #Content of the payload as an object 
                        {
                            "image": { #Passing an image, we put this before the text as best practise 
                                "format": "png", #Image format
                                "source": {"bytes": image_byte} #Passing image bytes 
                            }
                        },
                        {
                            "image": { #Passing an image, we put this before the text as best practise 
                                "format": "png", #Image format
                                "source": {"bytes": image_byte1} #Passing image bytes 
                            }
                        },
                        {
                            "text": prompt #indicating passing text, 'prompt' variable we will create in the cell below
                        }
                    ]
                }
            ],
            inferenceConfig={ # Inference parameters
                "temperature": temperature,
                "maxTokens": max_tokens,
                "topP": top_p
            }
            #additionalModelRequestFields={
            #}
        )
    except Exception as e: #Catch model invocation errors 
        print(e)
        result = "Model invocation error"
    try:
        result = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n' 
        return result
    except Exception as e: #Catching parsing errors 
        print(e)
        result = "Output parsing error"
    return result

### Inference 

In [75]:
#User Input
#Changed from Original:
#Darker ink in NEw York State text 
#ID changed 
#Issued date changed 
#-Anne added as middle name 
#Extra nub on the paw 
prompt = '''
Analyze the following documents given
The first document is our groundtruth, the second document given is under scruntiny. 
Compare & Contrast them, and find any discrepencies the second document may have.
In specific ensure to check & comment on these fields:
1/ First name 
2/ Middle name 
3/ Last name
4/ Issue date
5/ Expiry date
6/ ID Number
7/ Text color of the State Text
8/ Images on the bottom right

Also include the following for each document, provide them in <doc></doc> tags
1/classification 
2/extraction of all relevant data 
3/assessment of document from a fraudulent perspective - provide details such as specific features, conditions, image and data information as part of your assessment.
4/ Include confidence scores for steps 1- 3, each in precision .XXXX. Include your reasoning 
5/conform the extracted data into a JSON document and for upload to a backend system

'''

In [76]:
%%time 
#Calling the Converse API
response = invoke_bedrock_model(bedrock, modelId, prompt)
print(response)

<doc1>
Classification: Genuine
Extraction of relevant data:
- First name: Michelle
- Middle name: Marie
- Last name: Motorist
- Issue date: 03/07/2022
- Expiry date: 10/31/2029
- ID Number: 123 456 789
- Text color of the State Text: Blue
- Images on the bottom right: Bat, Maple Leaf, Paw Print

Assessment: This document appears to be a genuine New York State driver's license. The information on the license, including the name, address, date of birth, and other details, is consistent with a valid government-issued ID. The design elements, such as the color scheme, images, and security features, also match the expected characteristics of a legitimate driver's license.

Confidence scores:
1. Classification: 0.9999 - The document has all the hallmarks of a genuine New York State driver's license, and there are no obvious signs of forgery or tampering.
2. Extraction: 0.9999 - The relevant data fields are clearly visible and can be accurately extracted.
3. Assessment: 0.9999 - The document 